In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import json
from collections import Counter

In [2]:
data = pd.read_csv("sg_ca_data.csv", usecols=['county_fips','date_range_start','bucketed_distance_travelled'])

In [3]:
df = data[['county_fips','date_range_start','bucketed_distance_travelled']].copy()

In [4]:
df.head(10)

,county_fips,date_range_start,bucketed_distance_travelled
0,6001,2019-12-21 08:00:00+00:00,"{""16001-50000"":4,""0"":27,"">50000"":5,""2001-8000""..."
1,6019,2019-12-21 08:00:00+00:00,"{""16001-50000"":5,""0"":26,"">50000"":1,""2001-8000""..."
2,6021,2019-12-21 08:00:00+00:00,"{""16001-50000"":29,""0"":27,"">50000"":15,""2001-800..."
3,6029,2019-12-21 08:00:00+00:00,"{""16001-50000"":3,""0"":23,"">50000"":22,""2001-8000..."
4,6031,2019-12-21 08:00:00+00:00,"{""16001-50000"":17,""0"":23,"">50000"":14,""2001-800..."
5,6037,2019-12-21 08:00:00+00:00,"{""16001-50000"":6,""0"":17,"">50000"":5,""2001-8000""..."
6,6037,2019-12-21 08:00:00+00:00,"{""16001-50000"":16,""0"":41,"">50000"":12,""2001-800..."
7,6037,2019-12-21 08:00:00+00:00,"{""16001-50000"":8,""0"":19,"">50000"":5,""2001-8000""..."
8,6037,2019-12-21 08:00:00+00:00,"{""16001-50000"":7,""0"":14,"">50000"":2,""2001-8000""..."
9,6037,2019-12-21 08:00:00+00:00,"{""0"":29,"">50000"":17,""2001-8000"":13,""1-1000"":1,..."


In [5]:
df['COUNTYFP'] = df['county_fips'].apply(lambda x : str(x)[1:])

In [6]:
df['date_range_start'] = pd.to_datetime(df['date_range_start']) - pd.to_timedelta(7, unit='d')

In [7]:
template = Counter({"0":0,"1-1000":0,"1001-2000":0,"2001:8000":0,"8001-16000":0,"16001-50000":0,"<50000": 0})
df['bucketed_distance_travelled_counter'] = df['bucketed_distance_travelled'].apply(lambda x : template + Counter(json.loads(x)))

In [8]:
df = pd.DataFrame(df.groupby(['COUNTYFP', pd.Grouper(key='date_range_start', freq='W-MON')])['bucketed_distance_travelled_counter'].sum())

In [9]:
map = {'0':0, '1-1000':500, '1001-2000':1500, '2001-8000':5000, '8001-16000':12000, '16001-50000':33000, '>50000':50000}

def weighted_average(counter):
    dic = dict(counter)
    sigma, num = 0, 0
    for k,v in dic.items():
        sigma += map[k]*dic[k]
        num += map[k]
    return sigma/num

In [10]:
df['weighted_avg'] = df['bucketed_distance_travelled_counter'].apply(weighted_average)

In [11]:
df.drop('bucketed_distance_travelled_counter',inplace=True,axis=1)

In [12]:
df.head(10)

weighted_avg
COUNTYFP date_range_start                       
001      2019-12-16 00:00:00+00:00  25528.950980
         2019-12-23 00:00:00+00:00  65313.495098
         2019-12-30 00:00:00+00:00  54944.382353
         2020-01-06 00:00:00+00:00  47631.475490
         2020-01-13 00:00:00+00:00  46065.642157
         2020-01-20 00:00:00+00:00  41803.259804
         2020-01-27 00:00:00+00:00  43488.553922
         2020-02-03 00:00:00+00:00  47493.196078
         2020-02-10 00:00:00+00:00  49767.088235
         2020-02-17 00:00:00+00:00  47835.965686

In [13]:
import geopandas as gpd
import libpysal as lps
from shapely import speedups
from esda.moran import Moran
from esda.moran import Moran_Local
speedups.disable()

In [14]:
ca = gpd.read_file(filename="./CA_Counties/CA_Counties_TIGER2016.shp")

In [15]:
ca = ca[['COUNTYFP','geometry']].sort_values('COUNTYFP').reset_index(drop=True)

In [16]:
ca.head(10)

,COUNTYFP,geometry
0,001,"POLYGON ((-13612246.763 4538150.085, -13612346..."
1,003,"POLYGON ((-13366503.508 4678946.015, -13366492..."
2,005,"POLYGON ((-13472697.863 4647652.232, -13472697..."
3,007,"POLYGON ((-13565004.793 4798394.124, -13564991..."
4,009,"POLYGON ((-13428575.483 4627725.228, -13428534..."
5,011,"POLYGON ((-13589905.407 4781178.200, -13589880..."
6,013,"POLYGON ((-13610773.230 4565908.502, -13610791..."
7,015,"POLYGON ((-13838807.959 5120380.774, -13840421..."
8,017,"POLYGON ((-13482864.232 4681232.507, -13482880..."
9,019,"POLYGON ((-13325540.690 4439078.780, -13325502..."


In [17]:
wq = lps.weights.Queen.from_dataframe(df=ca,geom_col='geometry',idVariable='COUNTYFP')
wq.transform = 'r'

In [18]:
df = df.join(ca.set_index('COUNTYFP'), on='COUNTYFP').reset_index()

In [19]:
df[['lagged_weighted_avg','Moran I','LISA','LISA q','Moran p']] = ''

In [20]:
dates = df['date_range_start'].unique()

In [21]:
for date in dates:
    date_slice = df[df['date_range_start'] == date]
    y = date_slice['weighted_avg']
    ylag = lps.weights.lag_spatial(wq, y)
    mi = Moran(y,wq)
    mli = Moran_Local(y,wq)
    
    df.loc[df['date_range_start']==date,'lagged_weighted_avg'] = ylag
    df.loc[df['date_range_start']==date,'Moran I'] = mi.I
    df.loc[df['date_range_start']==date,'LISA'] = mli.Is
    df.loc[df['date_range_start']==date,'LISA q'] = mli.q
    df.loc[df['date_range_start']==date,'Moran p'] = mi.p_sim

In [22]:
df['significant'] = df['Moran p'].apply(lambda x : 1*(x<0.05))

In [23]:
df.drop('geometry',axis=1,inplace=True)

In [24]:
df['date_range_start'] = df['date_range_start'].apply(lambda x : str(x))

In [25]:
df.head(10)

,COUNTYFP,date_range_start,weighted_avg,lagged_weighted_avg,Moran I,LISA,LISA q,Moran p,significant
0,001,2019-12-16 00:00:00+00:00,25528.950980,15965.642974,0.390357,0.165009,1,0.001,1
1,001,2019-12-23 00:00:00+00:00,65313.495098,40743.438725,0.388815,0.193931,1,0.001,1
2,001,2019-12-30 00:00:00+00:00,54944.382353,35211.428922,0.406276,0.170086,1,0.001,1
3,001,2020-01-06 00:00:00+00:00,47631.475490,29524.928105,0.41075,0.13697,1,0.001,1
4,001,2020-01-13 00:00:00+00:00,46065.642157,29061.624183,0.427169,0.14396,1,0.001,1
5,001,2020-01-20 00:00:00+00:00,41803.259804,27713.006536,0.428373,0.134441,1,0.002,1
6,001,2020-01-27 00:00:00+00:00,43488.553922,28540.440359,0.424496,0.12826,1,0.001,1
7,001,2020-02-03 00:00:00+00:00,47493.196078,31354.585784,0.424463,0.131799,1,0.001,1
8,001,2020-02-10 00:00:00+00:00,49767.088235,33625.415033,0.429511,0.152243,1,0.001,1
9,001,2020-02-17 00:00:00+00:00,47835.965686,32769.132353,0.431044,0.156055,1,0.001,1


In [26]:
df.to_excel("processed.xlsx")